## Final Project Part 3

<b>Syeda Rija Zaineb</b>

In [ ]:
myJekyllSaveDir = '/Users/rijaz/szaineb.github.io/assets/json/'

In [ ]:
import altair as alt
import requests
import pandas as pd
import geopandas
import json

In [ ]:
#reading police precincts data
nyPolice = geopandas.read_file("https://raw.githubusercontent.com/szaineb/szaineb.github.io/master/data/Police_Precincts.geojson")

In [ ]:
# nyPolice

In [ ]:
#reading crimes data
crimes = pd.read_csv("https://raw.githubusercontent.com/szaineb/szaineb.github.io/master/data/NYPD%20Hate%20Crimes.csv")

In [ ]:
# crimes

In [ ]:
#had to do this to make line 13 work
nyPolice['precinct']=nyPolice['precinct'].astype(int)
crimes['Complaint Precinct Code']=crimes['Complaint Precinct Code'].astype(int)

In [ ]:
#merging precinct and crimes data on precinct
mergedRes = pd.merge(crimes, nyPolice, how='left', left_on =['Complaint Precinct Code'], right_on=['precinct'])

In [ ]:
#dropped extra precinct column
mergedRes=mergedRes.drop(columns = 'Complaint Precinct Code')

In [ ]:
#converting dataframe to geodataframe
mergedGeo = geopandas.GeoDataFrame(mergedRes)

In [ ]:
#adding complaint count column in dataframe
mergedGeo["complaint_count_precinct"] = mergedGeo.groupby(['precinct', 'Complaint Year Number'])['Full Complaint ID'].transform('count')

In [ ]:
# mergedGeo.groupby('Offense Description')['Full Complaint ID'].count()

In [ ]:
mergedGeo2021 = mergedGeo[mergedGeo["Complaint Year Number"] == 2021]

In [ ]:
mergedGeo2021 = mergedGeo2021.rename(columns = {'Bias Motive Description':'Bias_Motive_Description'})

In [ ]:
# mergedGeo2021

In [ ]:
# mergedGeo2021['Patrol Borough Name'].unique()

In [ ]:
mergedGeo2021.replace("PATROL BORO MAN SOUTH", 
           "Manhattan South", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO MAN NORTH", 
           "Manhattan North", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO BRONX", 
           "Bronx", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO BKLYN SOUTH", 
           "Brooklyn South", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO BKLYN NORTH", 
           "Brooklyn North", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO QUEENS SOUTH", 
           "Queens South", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO QUEENS NORTH", 
           "Queens North", 
           inplace=True)

mergedGeo2021.replace("PATROL BORO STATEN ISLAND", 
           "Staten Island", 
           inplace=True)

In [ ]:
# mergedGeo2021['Patrol Borough Name'].unique()

In [ ]:
mergedGeo2021['Patrol Borough Name'] = mergedGeo2021['Patrol Borough Name'].str.upper()

In [ ]:
# mergedGeo2021['Patrol Borough Name'].unique()

<b>First NYC Bar Chart</b>

In [ ]:
bar_base = alt.Chart(mergedGeo2021).mark_bar().encode(
    x=alt.X('count:Q', axis=None),
    y=alt.Y('Bias_Motive_Description:N',
        sort=alt.EncodingSortField(field='count', order='descending', op='sum'), axis=alt.Axis(title=None)
    ),
    tooltip=alt.Tooltip('count:Q')
).transform_aggregate(
    count='count()',
    groupby=['Bias_Motive_Description']
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'count', 'order': 'descending'}]
).transform_filter('datum.rank <= 5'
).properties(title="Top 5 Bias Motivations behind Hate Crimes in New York City", width=600, height=100)

color_bar = bar_base.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='reds'),legend=None),
    opacity=alt.value(0.7)
)

bar_label = bar_base.mark_text(
    align='right',
    baseline='middle',
    dx=20,
).encode(
    text='count:Q'
)

bar_bias_ny = color_bar + bar_label

In [ ]:
bar_bias_ny

In [ ]:
bar_bias_ny.save(myJekyllSaveDir+'NYPD_Bias_Motives_Top5.json')

<b>Second NYC Bar Chart</b>

In [ ]:
bar_base_ny_boroughs = alt.Chart(mergedGeo2021).mark_bar().encode(
    x=alt.X('count:Q', axis=None),
    y=alt.Y('Patrol Borough Name:N',
        sort=alt.EncodingSortField(field='count', order='descending', op='sum'), axis=alt.Axis(title=None)
    ),
    tooltip=alt.Tooltip('count:Q')
).transform_aggregate(
    count='count()',
    groupby=['Patrol Borough Name']
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'count', 'order': 'descending'}]
).properties(title="NYPD Patrol Boroughs by Count of Hate Crimes", width=550, height=200)

color_bar_ny_boroughs = bar_base_ny_boroughs.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='reds', reverse=True),legend=None),
    opacity=alt.value(0.7)
)

bar_label_ny_boroughs = bar_base_ny_boroughs.mark_text(
    align='right',
    baseline='middle',
    dx=20,
).encode(
    text='count:Q'
)

bar_crimes_ny_boroughs = color_bar_ny_boroughs + bar_label_ny_boroughs

In [ ]:
bar_crimes_ny_boroughs

In [ ]:
bar_crimes_ny_boroughs.save(myJekyllSaveDir+'NYPD_Patrol_Boroughs.json')

<b>NYC Dashboard - Main Interactive Visualization</b>

In [ ]:
## Precinct Map

multi = alt.selection_multi(fields=['complaint_count_precinct', 'precinct'], bind='legend')
color = alt.condition(multi,
                  alt.Color('complaint_count_precinct',type='quantitative',bin=alt.Bin(step=5), legend={'direction':"horizontal", 'orient':"top-left"},
                  scale=alt.Scale(scheme='yelloworangered'), title='Complaint Count'),
                  alt.value('lightgray'))

# map
precinct_map = alt.Chart(mergedGeo2021, title = 'Hate Crime Complaint Filings in New York by Precincts').mark_geoshape(
    stroke='black',
    strokeWidth=1
).encode(
    color=color,
    tooltip=[alt.Tooltip('precinct', title='Precinct'),alt.Tooltip('Patrol Borough Name', title ='Patrol Borough'),alt.Tooltip('complaint_count_precinct', title ="No. of complaints")]
).properties(
    height=500
).add_selection(
    multi
)

# precinct_map

In [ ]:
## Bar Chart

hist_base = alt.Chart(mergedGeo2021, title = 'Hate Crime Complaint Filings in NYC by Bias Motivation').encode(
    alt.X("Bias_Motive_Description:N",axis=alt.Axis(title=None, labelLimit=200)),
    alt.Y("count()", scale=alt.Scale(domain=[0,220]), axis=alt.Axis(title='No. of Complaints')),
#     alt.Tooltip("count()")
).transform_filter(
    multi
)

hist = hist_base.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='goldred'),legend=None)
)

text_mark = hist_base.mark_text(
    align='center',
    baseline='middle',
    dy=-5,
).encode(
    text='count()'
)

motive_hist = hist + text_mark

In [ ]:
dashboard = precinct_map.properties(width=400) | motive_hist.properties(width=400)
dashboard

In [ ]:
dashboard.save(myJekyllSaveDir+'NYPD_hate_crime_dashboard.json')

<b>Contextual Visualizations - Chicago Hate Crimes</b>

In [ ]:
chicago_crimes = pd.read_csv("https://raw.githubusercontent.com/szaineb/szaineb.github.io/master/data/ChicagoData.csv")

In [ ]:
chicago_crimes

In [ ]:
chicago_crimes['Year'] = pd.to_datetime(chicago_crimes['DATE']).dt.year

In [ ]:
# chicago_crimes['Year']

In [ ]:
chicago_crimes_2021 = chicago_crimes[chicago_crimes['Year'] == 2021]

In [ ]:
# chicago_crimes_2021.groupby('Community').count()

<b>Top 5 Bias Motives in Chicago 2021</b>

In [ ]:
bar_base_chicago = alt.Chart(chicago_crimes_2021).mark_bar().encode(
    x=alt.X('count:Q', axis=None),
    y=alt.Y('Motivation:N',
        sort=alt.EncodingSortField(field='count', order='descending', op='sum'), axis=alt.Axis(title=None)
    ),
    tooltip=alt.Tooltip('count:Q')
).transform_aggregate(
    count='count()',
    groupby=['Motivation']
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'count', 'order': 'descending'}]
).transform_filter('datum.rank <= 5'
).properties(title="Top 5 Motives behind Hate Crimes in Chicago", width=550, height=100)

color_bar_chicago = bar_base_chicago.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='teals'),legend=None),
    opacity=alt.value(0.7)
)

bar_label_chicago = bar_base_chicago.mark_text(
    align='right',
    baseline='middle',
    dx=18,
).encode(
    text='count:Q'
)

bar_bias_chicago = color_bar_chicago + bar_label_chicago

In [ ]:
bar_bias_chicago

In [ ]:
bar_bias_chicago.save(myJekyllSaveDir+'Chicago_Top5_Motives.json')

In [ ]:
bar_base_chicago_boroughs = alt.Chart(chicago_crimes_2021).mark_bar().encode(
    x=alt.X('count:Q', axis=None),
    y=alt.Y('Community:N',
        sort=alt.EncodingSortField(field='count', order='descending', op='sum'), axis=alt.Axis(title=None)
    ),
    tooltip=alt.Tooltip('count:Q')
).transform_aggregate(
    count='count()',
    groupby=['Community']
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'count', 'order': 'descending'}]
).transform_filter('datum.rank <= 5'
).properties(title="Top 5 Communities with most Hate Crimes in Chicago", width=550, height=100)

color_bar_chicago_boroughs = bar_base_chicago_boroughs.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='teals'),legend=None),
    opacity=alt.value(0.7)
)

bar_label_chicago_boroughs = bar_base_chicago_boroughs.mark_text(
    align='right',
    baseline='middle',
    dx=18,
).encode(
    text='count:Q'
)

bar_bias_chicago_communities = color_bar_chicago_boroughs + bar_label_chicago_boroughs

In [ ]:
bar_bias_chicago_communities

In [ ]:
bar_bias_chicago_communities.save(myJekyllSaveDir+'Chicago_Top5_Communities.json')

<b>Write-up</b>

In this notebook, I have developed all the visualizations published in my article <a href="https://szaineb.github.io/projects/final-project">here</a>.

I used public data from New York Police Department (NYPD) and Chicago Police Department (CPD) to create all these visualizations. I have uploaded all these data sets on my github repository and also added a link to them in my article.

In the description here and in the article, I have interchangeably used complaints filed and hate crimes terms to represent the same thing.

I have first developed 2 bar charts using the merged data from two datasets 'Police_Precincts.geojson' and 'NYPD Hate Crimes.csv'. In the first chart, I presented Top 5 bias motivations for hate crimes in NYC. In the second chart, I showed the patrol borough sorted by count of hate crime incidents. I used bar chart for as I am comparing categorical variables here.

Then I have an interactive dashboard which gives an overview of the hate crime complaints filed in various NYC police precincts during the year 2021. By using this dashboard, users can see the number of complaints filed across precincts, the areas that had the least and most crimes as well as the biases that motivated hate crimes in NYC in the year 2021.

This dashboard has two charts. On the left you can find the NYC map based on police precincts. The map is color coded by the number of hate crime complaints filed in 2021. I used yelloworangered color scheme for this. Lighter yellow shade shows less number of crimes while brightest red color depicts highest number of crime complaints. On the right, there is a bar chart that shows count of complaints by 'Bias Motive Description'. If we look at the overall stats, we see that majority of the crimes had anti-jewish motives followed by anti-asian.

For contextual visualizations, I chose to analyze Chicago hate crimes as NYC and Chicago city are often compared for crime rates. I used 'ChicagoData.csv' that I got from Chicago Police Department's website. I created 2 bar charts using this file. The first chart displays top 5 bias motivations for hate crimes in Chicago and the second one shows top 5 communities with most incidents. Code for contextual visualizations can be fo

I have tried to follow a consitent design theme. I used similar charts so that comparison is easy to understand. Similarly, I followed orange, red theme for NYC as we are talking about crimes so red seemed appropriate. I followed same color scheme for all NYC charts to ensure that it is easy for users to follow that all this information is some how related. To differentiate Chicago from NYC, I used teals color scheme for its charts.

<b>Data Sources</b>

Police Precincts. (n.d.). NYC Open Data. https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz

NYPD Hate Crimes | NYC Open Data. (2022, October 19). https://data.cityofnewyork.us/Public-Safety/NYPD-Hate-Crimes/bqiq-cu78

Hate Crime Dashboard. (n.d.). Chicago Police Department. https://home.chicagopolice.org/statistics-data/data-dashboards/hate-crime-dashboard/